## Functions

In [95]:
from anchor import utils

import time
import csv
import newlime_utils

from tabulate import tabulate

def sample(index, dataset, dataset_name, print_info=True, write_file=False):
    trg, label, tab = newlime_utils.get_trg_sample(index, dataset)
    if print_info:
        print('Prediction:', dataset.class_names[rf.predict(trg.reshape(1, -1))[0]])
        print('True:      ', dataset.class_names[dataset.labels_test[index]])
        print(tabulate(tab))
    if write_file:
        with open('img/%s/%05d-instance.csv' % (dataset_name, index), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows([['feature', 'value']])
            writer.writerows(tab)
    return trg, label

In [2]:
from lime import explanation
from lime import lime_tabular

def lime_original(trg, pred_label):
    lime_explainer = lime_tabular.LimeTabularExplainer(
        dataset.train,
        feature_names=dataset.feature_names,
        class_names=dataset.class_names,
        discretize_continuous=False)
    lime_exp = lime_explainer.explain_instance(
        trg, rf.predict_proba, num_features=5, top_labels=1)
    # lime_exp.show_in_notebook(show_table=True, show_all=True)
    weights = [0] * len(dataset.feature_names)
    for t in lime_exp.local_exp[pred_label]:
        weights[t[0]] = t[1] * (pred_label * 2 - 1)
    newlime_utils.plot_weights(weights, dataset.feature_names)

In [3]:
from anchor import anchor_tabular

def anchor_original(trg, threshold=0.85):
    anchor_explainer = anchor_tabular.AnchorTabularExplainer(
        dataset.class_names,
        dataset.feature_names,
        dataset.train,
        dataset.categorical_names)
    anchor_exp = anchor_explainer.explain_instance(
        trg, rf.predict, threshold)

    print('Threshold:  %.2f' % threshold)
    # print('Prediction:', anchor_explainer.class_names[rf.predict(trg.reshape(1, -1))[0]])
    print('Anchor:     %s' % (' AND '.join(anchor_exp.names())))
    print('Precision:  %.2f' % anchor_exp.precision())
    print('Coverage:   %.2f' % anchor_exp.coverage())

In [116]:
import newlime_base
import newlime_tabular
import newlime_utils

def new_lime(trg, dataset, threshold, epsilon, beam_size, my_verbose, print_info=True):
    if print_info:
        print("-----------")
        print("Threshold: ", threshold)
    anchor_explainer = newlime_tabular.NewLimeTabularExplainer(
        dataset.class_names,
        dataset.feature_names,
        dataset.train,
        dataset.categorical_names
    )
    hyper_param = newlime_tabular.HyperParam(
        delta = 0.05,
        epsilon = epsilon, 
        epsilon_stop = 0.05,
        beam_size = beam_size,
        batch_size = 100,
        desired_confidence = threshold,
        coverage_samples_num = 10000,
        max_rule_length = None,
    )
    result = anchor_explainer.my_explain_instance(
        trg, rf.predict, hyper_param)

    if result is None:
        return None
        
    anchor_exp, surrogate_model = result
    names = anchor_exp.names()
    multiline_names = []
    max_i = int(len(names) / 3)
    for i in range(max_i):
        triple = [names[i * 3], names[i * 3 + 1], names[i * 3 + 2]]
        multiline_names.append(' AND '.join(triple))
    if len(names) != max_i* 3:
        multiline_names.append(' AND '.join(names[max_i * 3:]))

    weights = list(surrogate_model['LogisticRegression'].weights.values())
    feat = dataset.feature_names
    rule = ' AND \n'.join(multiline_names)
    prec = anchor_exp.precision()
    cov = anchor_exp.coverage()
    if print_info:
        print("Rule     : ", rule)
        print("Precision: ", prec)
        print("Coverage : ", cov)
    
    if weights == []:
        print("Error! Surrogate model has no weights!")
    elif print_info:
        newlime_utils.plot_weights(weights, feat, rule, prec, cov)
    return rule, prec, cov

## Generating Explanations

In [20]:
dataset_folder = 'anchor-experiments/datasets/'
dataset_name = 'recidivism'
# dataset_name = 'adult'
# dataset_name = 'diabetes'
# dataset_name = 'default'
# dataset_name = 'lending'

# dataset = utils.load_dataset(
    # dataset_name, balance=True, dataset_folder=dataset_folder, discretize=True)

dataset = newlime_utils.load_dataset(dataset_name, dataset_folder, balance=True)
print(dataset.class_names)

print(dataset.train.shape[0])
print(dataset.test.shape[0])
print(dataset.train.shape[0] + dataset.test.shape[0])

['No more crimes', 'Re-arrested']
5635
705
6340


In [6]:
import sklearn.ensemble

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
rf.fit(dataset.train, dataset.labels_train)

print('Train', sklearn.metrics.accuracy_score(
    dataset.labels_train, rf.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(
    dataset.labels_test, rf.predict(dataset.test)))

Train 0.935226264418811
Test 0.7531914893617021


In [7]:
import random

i = random.randint(10, dataset.test.shape[0])

# adult
# 730, 956, 1196, 1522
# i = 1443 # --- Capital Gain = 2
i = 0

# recidivism
# i = 266
# i = 599

trg, trg_label = sample(i, dataset, dataset_name, dataset_folder)

Prediction: Re-arrested
True:       Re-arrested
----------------------  -------------------------------
Race                    White (1)
Alcohol                 No (0)
Junky                   No (0)
Supervised Release      Yes (1)
Married                 No (0)
Felony                  Yes (1)
WorkRelease             Yes (1)
Crime against Property  Yes (1)
Crime against Person    No (0)
Gender                  Male (1)
Priors                  1
YearsSchool             8.00 < YearsSchool <= 10.00 (1)
PrisonViolations        0
Age                     Age <= 21.00 (0)
MonthsServed            4.00 < MonthsServed <= 9.00 (1)
Recidivism              Re-arrested (1)
----------------------  -------------------------------


In [8]:
import importlib
importlib.reload(newlime_base)
importlib.reload(newlime_tabular)
importlib.reload(newlime_utils)
write_file = False

i = random.randint(10, dataset.test.shape[0])
i = 0
trg, trg_label = sample(i, dataset, dataset_name, dataset_folder)

for t in [0.70, 0.80, 0.90, 0.95]:
    new_lime(trg, dataset, threshold=t, beam_size=1, epsilon=0.1, my_verbose=True)
    if write_file:
        img_file = 'img/%s/%05d-%03d.png' % (dataset_name, i, t * 100)
        plt.savefig(img_file, bbox_inches='tight')

Prediction: Re-arrested
True:       Re-arrested
----------------------  -------------------------------
Race                    White (1)
Alcohol                 No (0)
Junky                   No (0)
Supervised Release      Yes (1)
Married                 No (0)
Felony                  Yes (1)
WorkRelease             Yes (1)
Crime against Property  Yes (1)
Crime against Person    No (0)
Gender                  Male (1)
Priors                  1
YearsSchool             8.00 < YearsSchool <= 10.00 (1)
PrisonViolations        0
Age                     Age <= 21.00 (0)
MonthsServed            4.00 < MonthsServed <= 9.00 (1)
Recidivism              Re-arrested (1)
----------------------  -------------------------------
-----------
Threshold:  0.7
Rule     :  Felony = Yes AND Priors = 1
Precision:  0.7096897740329375
Coverage :  0.2041
-----------
Threshold:  0.8


KeyboardInterrupt: 

In [ ]:
lime_original(trg, rf.predict([dataset.test[i]])[0])

write_file = False
if write_file:
    img_file = 'img/%s/%05d-LIME.png' % (dataset_name, i)
    plt.savefig(img_file, bbox_inches='tight')

In [ ]:
i

In [ ]:
anchor_original(trg, threshold=0.80)

## Imbalanced Data

### Check balance of datasets

In [21]:
import importlib
importlib.reload(newlime_base)
importlib.reload(newlime_tabular)
importlib.reload(newlime_utils)

for dataset_name in ["adult", "recidivism", "lending"]:
    print(dataset_name)
    balanced = newlime_utils.load_dataset(dataset_name, dataset_folder, balance=True)
    imbalanced = newlime_utils.load_dataset(dataset_name, dataset_folder, balance=False)
    for d, s in zip([balanced, imbalanced], ["Balanced:   ", "Imbalanced: "]):
        label_count = [0, 0]
        for l in [int(x) for x in d.labels_train.tolist()]:
            label_count[l] += 1
        print(s, label_count)
    print()

adult
Balanced:    [6264, 6281]
Imbalanced:  [19694, 6354]

recidivism
Balanced:    [2825, 2810]
Imbalanced:  [4801, 2838]

lending
Balanced:    [4502, 4568]
Imbalanced:  [4502, 4568]



### Undersampling

In [97]:
import numpy as np

dataset_folder = 'anchor-experiments/datasets/'
dataset_name = 'recidivism'
dataset = newlime_utils.load_dataset(dataset_name, dataset_folder, balance=False)

print(dataset.train.shape)
print(dataset.labels_train.shape)
print(dataset.train_idx.shape)

print(dataset.validation.shape)
print(dataset.labels_validation.shape)
print(dataset.validation_idx.shape)

print(dataset.test.shape)
print(dataset.labels_test.shape)
print(dataset.test_idx.shape)

# get indexes of data rows that will be deleted
minor_class = 1
minor_data_num = 80
minor_data_idx = []
for i, l in enumerate(dataset.labels):
    if l == minor_class:
        minor_data_idx.append(i)
np.random.seed(1024)
del_list = np.random.choice(
    minor_data_idx,
    len(minor_data_idx) - minor_data_num,
    replace=False,
)

del_train_list = []
for i, x in enumerate(dataset.train_idx):
    if x in del_list:
        del_train_list.append(i)

del_valid_list = []
for i, x in enumerate(dataset.validation_idx):
    if x in del_list:
        del_valid_list.append(i)

del_test_list = []
for i, x in enumerate(dataset.test_idx):
    if x in del_list:
        del_test_list.append(i)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_train.tolist()]:
    label_count[l] += 1
print(label_count)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_validation.tolist()]:
    label_count[l] += 1
print(label_count)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_test.tolist()]:
    label_count[l] += 1
print(label_count)

dataset.data = np.delete(dataset.data, del_list, axis=0)
dataset.labels = np.delete(dataset.labels, del_list, axis=0)

dataset.train = np.delete(dataset.train, del_train_list, axis=0)
dataset.labels_train = dataset.data = np.delete(dataset.labels_train, del_train_list, axis=0)
dataset.train_idx = np.delete(dataset.train_idx, del_train_list, axis=0)

dataset.validation = np.delete(dataset.validation, del_valid_list, axis=0)
dataset.labels_validation = np.delete(dataset.labels_validation, del_valid_list, axis=0)
dataset.validation_idx = np.delete(dataset.validation_idx, del_valid_list, axis=0)

dataset.test = np.delete(dataset.test, del_test_list, axis=0)
dataset.labels_test = np.delete(dataset.labels_test, del_test_list, axis=0)
dataset.test_idx = np.delete(dataset.test_idx, del_test_list, axis=0)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_train.tolist()]:
    label_count[l] += 1
print(label_count)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_validation.tolist()]:
    label_count[l] += 1
print(label_count)

label_count = [0, 0]
for l in [int(x) for x in dataset.labels_test.tolist()]:
    label_count[l] += 1
print(label_count)

print(dataset.train.shape)
print(dataset.labels_train.shape)
print(dataset.train_idx.shape)

print(dataset.validation.shape)
print(dataset.labels_validation.shape)
print(dataset.validation_idx.shape)

print(dataset.test.shape)
print(dataset.labels_test.shape)
print(dataset.test_idx.shape)

(7639, 15)
(7639,)
(7639,)
(955, 15)
(955,)
(955,)
(955, 15)
(955,)
(955,)
[4801, 2838]
[594, 361]
[632, 323]
[4801, 62]
[594, 10]
[632, 8]
(4863, 15)
(4863,)
(4863,)
(604, 15)
(604,)
(604,)
(640, 15)
(640,)
(640,)


In [117]:
import importlib
importlib.reload(newlime_base)
importlib.reload(newlime_tabular)
importlib.reload(newlime_utils)

t = 0.8
num_calc = 1000

covs = []
for i in range(num_calc):
    trg, trg_label = sample(None, dataset, dataset_name, print_info=False)
    result = new_lime(
        trg, 
        dataset, 
        threshold=t, 
        beam_size=1, 
        epsilon=0.1, 
        my_verbose=True, 
        print_info=False
    )
    if result is None:
        print("None")
        i -= 1
        continue
    _, _, cov = result
    covs.append(cov)
    print(cov)
    print(i + 1, "/", num_calc)
    
print(np.average(covs))
print(np.std(covs))

0.0038
1 / 1000
0.3656
2 / 1000
0.0756
3 / 1000
0.1639
4 / 1000
0.0788
5 / 1000
0.3691
6 / 1000
0.0065
7 / 1000
0.1406
8 / 1000
0.0858
9 / 1000
0.2228
10 / 1000
0.0792
11 / 1000
0.2127
12 / 1000
0.0022
13 / 1000
0.0156
14 / 1000
0.0072
15 / 1000
0.2146
16 / 1000
0.3569
17 / 1000
0.0763
18 / 1000
0.0094
19 / 1000
0.0457
20 / 1000
0.3605
21 / 1000
0.0407
22 / 1000
0.0852
23 / 1000
0.0471
24 / 1000
0.1233
25 / 1000
0.1222
26 / 1000
0.0703
27 / 1000
0.0454
28 / 1000
0.0862
29 / 1000
0.0751
30 / 1000
0.041
31 / 1000
0.2168
32 / 1000
0.0865
33 / 1000
0.0925
34 / 1000
0.0816
35 / 1000
0.1514
36 / 1000
0.0734
37 / 1000
0.2309
38 / 1000
0.2192
39 / 1000
0.0268
40 / 1000
0.1245
41 / 1000
0.0104
42 / 1000
0.0206
43 / 1000
0.0798
44 / 1000
0.079
45 / 1000
0.0352
46 / 1000
0.1225
47 / 1000
0.08
48 / 1000
0.0
49 / 1000
0.0796
50 / 1000
0.1272
51 / 1000
0.0638
52 / 1000
0.3687
53 / 1000
0.2214
54 / 1000
0.0918
55 / 1000
0.0392
56 / 1000
0.194
57 / 1000
0.1145
58 / 1000
0.0845
59 / 1000
0.2302
60 / 10

In [118]:
balanced = newlime_utils.load_dataset(dataset_name, dataset_folder, balance=True)

covs = []
for i in range(num_calc):
    trg, trg_label = sample(None, balanced, dataset_name, print_info=False)
    result = new_lime(
        trg, 
        balanced, 
        threshold=t, 
        beam_size=1, 
        epsilon=0.1, 
        my_verbose=True, 
        print_info=False,
    )
    if result is None:
        print("None")
        i -= 1
        continue
    _, _, cov = result
    covs.append(cov)
    print(cov)
    print(i + 1, "/", num_calc)
print(np.average(covs))
print(np.std(covs))

0.0321
1 / 1000
0.0044
2 / 1000
0.0418
3 / 1000
0.0295
4 / 1000
0.0019
5 / 1000
0.0119
6 / 1000
0.0055
7 / 1000
0.0091
8 / 1000
0.0422
9 / 1000
0.0038
10 / 1000
0.0393
11 / 1000
0.0252
12 / 1000
0.0036
13 / 1000
0.0155
14 / 1000
0.0076
15 / 1000
0.0469
16 / 1000
0.0234
17 / 1000
0.0506
18 / 1000
0.0721
19 / 1000
0.0004
20 / 1000
0.0163
21 / 1000
0.0034
22 / 1000
0.0105
23 / 1000
0.0272
24 / 1000
0.008
25 / 1000
0.0706
26 / 1000
0.0085
27 / 1000
0.0111
28 / 1000
0.0053
29 / 1000
0.012
30 / 1000
0.0056
31 / 1000
0.0009
32 / 1000
0.003
33 / 1000
0.0149
34 / 1000
0.0059
35 / 1000
0.0067
36 / 1000
0.0042
37 / 1000
0.0199
38 / 1000
0.0001
39 / 1000
0.0028
40 / 1000
0.0052
41 / 1000
0.0093
42 / 1000
0.0236
43 / 1000
0.0113
44 / 1000
0.0012
45 / 1000
0.0258
46 / 1000
0.0045
47 / 1000
0.064
48 / 1000
0.0079
49 / 1000
0.0058
50 / 1000
0.013
51 / 1000
0.0067
52 / 1000
0.0015
53 / 1000
0.0028
54 / 1000
0.0456
55 / 1000
0.0294
56 / 1000
0.0092
57 / 1000
0.0052
58 / 1000
0.028
59 / 1000
0.0006
60 / 